In [4]:
import pandas as pd
import sqlite3
import panel as pn
from datetime import datetime, date
import param

In [5]:
def fetch_data(query, db_path='db/database.db'):
    """
    Fetch data from the SQLite database and return it as a pandas DataFrame.

    :param query: SQL query to execute.
    :param db_path: Path to the SQLite database file. Default is 'database.db'.
    :return: pandas DataFrame containing the query results.
    """
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    
    # Use pandas to execute the query and fetch the data
    df = pd.read_sql_query(query, conn)
    
    # Close the connection
    conn.close()
    
    return df

In [6]:
query = """
select 
transactions.id,
persons.lastname,
persons.firstname,
transactions.asset,
transactions.ticker,
transactions.security,
transactions.transaction_type,
transactions.transaction_date,
transactions.notification_date,
transactions.min_amount,
transactions.max_amount,
transactions.docid,
fillings.filing_type,
fillings.filing_date
from 
transactions 
inner join fillings on fillings.docid = transactions.docid
inner join persons on fillings.person_id = persons.id
"""

df = fetch_data(query)

In [7]:
# some processing

# parse dates
df['notification_date'] = pd.to_datetime(df['notification_date'], format='%m/%d/%Y', errors='coerce')
df['transaction_date'] = pd.to_datetime(df['transaction_date'], format='%m/%d/%Y', errors='coerce')
df['filing_date'] = pd.to_datetime(df['filing_date'], format='%m/%d/%Y', errors='coerce')

# parse amounts
df['min_amount'] = df['min_amount'].str.replace(',', '').astype(float)
df['max_amount'] = df['max_amount'].str.replace(',', '').astype(float)

# join names
df['name'] = df['firstname'] + " " + df['lastname']
df.drop(columns=['firstname','lastname'], inplace=True)

In [17]:
# drop down options
names = list(df['name'].unique())
securities = list(df['security'].unique())
tickers = list(df['ticker'].unique())
buysell = ['Buy', 'Sell']
names.append('-')
securities.append('-')
tickers.append('-')
buysell.append('-')

# Define the FilterParams class
class FilterParams(param.Parameterized):
    start_date = param.CalendarDate(default=date(2020, 1, 1))
    end_date = param.CalendarDate(default=date(2025, 12, 31))
    selected_name = param.Selector(objects=names, default='-')
    security = param.Selector(objects=securities, default='-')
    ticker = param.Selector(objects=tickers, default='-')
    action = param.ObjectSelector(default='-', objects=buysell)

filter_params = FilterParams()

# Create the widgets using Param
name_dropdown = pn.Param(filter_params.param.selected_name)
security_dropdown = pn.Param(filter_params.param.security)
ticker_dropdown = pn.Param(filter_params.param.ticker)
start_date_picker = pn.Param(filter_params.param.start_date)
end_date_picker = pn.Param(filter_params.param.end_date)
select_widget = pn.Param(filter_params.param.action)

In [18]:
PAGE_SIZE = 35
WIDTHS = {
        'asset': 100,
        'description': 250,
        'amount': 100,
        'tag': 100,
        'card': 100,
        'category': 200
    }

@pn.depends(filter_params.param.selected_name, filter_params.param.security, filter_params.param.ticker, filter_params.param.start_date, filter_params.param.end_date, filter_params.param.action)
def filter_df(selected_name, security, ticker, start_date, end_date, action):
    filtered_df = df.copy()

    if selected_name and selected_name != '-':
        filtered_df = filtered_df[filtered_df['name'] == selected_name]

    if security and security != '-':
        filtered_df = filtered_df[filtered_df['security'] == security]

    if ticker and ticker != '-':
        filtered_df = filtered_df[filtered_df['ticker'] == ticker]

    if action and action != '-':
        if action == 'Buy':
            filtered_df = filtered_df[filtered_df['transaction_type'] == 'P']
        else:
            filtered_df = filtered_df[filtered_df['transaction_type'] != 'P']

    if start_date and end_date:
        mask = (filtered_df['transaction_date'] >= pd.to_datetime(start_date)) & (filtered_df['transaction_date'] <= pd.to_datetime(end_date))
        filtered_df = filtered_df[mask]
        
    
    return pn.widgets.Tabulator(filtered_df, pagination='local', page_size=PAGE_SIZE, sizing_mode='stretch_width', show_index=False)

In [19]:
# Display the layout
layout = pn.Column(
    pn.Row(name_dropdown, security_dropdown, ticker_dropdown),
    pn.Row(start_date_picker, end_date_picker),
    select_widget,
    pn.panel(filter_df)
)

layout.servable()
layout.show()

Launching server at http://localhost:41371


Opening in existing browser session.
